# Scenemixer
Tool for combining two photographs into a composite image (center + surrounding).

In [1]:
from __future__ import division
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tools import *
%matplotlib inline

### Settings

In [2]:
image_folder = "./images/"  # must end with /
# filenames in []
image_in     = ["lapland.jpeg", "sarka.jpeg", "helsinki.jpeg"]  
                                                
image_out    = image_in     # use the same as image_in
new_size     = (1024, 1024) # in pixels (width, height)
radius       = 400          # radius of central/inside area
ring_width   = 70
ring_colour  = (128, 128, 128)
blur         = 25

In [3]:
fn_in  = ["%s%s" % (image_folder, i) for i in image_in]
fn_out = ["%s%s" % (image_folder, i) for i in image_out]

img_in  = [crop_and_resize_image(
        cv2.imread(i, cv2.CV_LOAD_IMAGE_COLOR), new_size, grayscale = True) for i in fn_in]
img_out = [crop_and_resize_image(
        cv2.imread(i, cv2.CV_LOAD_IMAGE_COLOR), new_size, grayscale = True) for i in fn_out]
ring    = add_ring(img_in[0], radius, width = ring_width, colour = ring_colour, blur = blur)
alpha   = ring[:,:,3]
alpha   = alpha.reshape((alpha.shape[0], alpha.shape[1], 1)) / 255.


for iin in np.arange(len(image_in)):
    for iout in np.arange(len(image_out)):
        fn  = "output/%03d_%03d.jpeg" % (iin, iout)
        im  = combine_images(img_in[iin], img_out[iout], radius)
        if ring_width == 0:
            imf = im
        else:
            imf = im[:,:,:3] * (1 - alpha) + ring[:,:,:3] * alpha + 0
        cv2.imwrite(fn, imf)
    